In [1]:
import os

import dotenv
import pandas as pd
import pymongo

In [2]:
dotenv.load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")
DBclient = pymongo.MongoClient(mongodb_uri)
db = DBclient["ML2-Abschlussprojekt"]

# Base Data

In [3]:
df_base = pd.read_excel("Student-Data.xlsx", "Base_Data").convert_dtypes().replace({pd.NA:None})
df_base.head()

,firstname,lastname,gender,birthday,school_entry,diagnose,compensation,class_teacher,special_education_teacher
0,Bruce,Banner,männlich,2009-02-02,2022-08-01,Schizophrenie,Separater Raum bei Prüfungen,Nick Fury,Miss Marvel
1,Clint,Barton,männlich,2009-04-04,2020-08-01,Autismus-Spektrums-Störung,"Separater Raum bei Prüfungen, mehr Zeit bei Pr...",Nick Fury,Miss Marvel
2,Natasha,Romanoff,weiblich,2009-03-03,2022-08-01,None,None,Nick Fury,Miss Marvel
3,Steve,Rogers,männlich,2009-01-01,2020-08-01,None,None,Nick Fury,Miss Marvel
4,Tony,Stark,männlich,2009-01-01,2020-08-01,Narzistische Persönlichkeitsstörung,None,Nick Fury,Miss Marvel


In [4]:
base_data = [
    row.to_dict() for col, row in df_base.iterrows()
]

In [5]:
collection = db["Base_Data"]
result = collection.insert_many(base_data)

# Mapping: Prename to ObjectIDs from MongoDB
student_map = {
    s["firstname"]: r
    for s,r in zip(base_data, result.inserted_ids)
}

In [6]:
student_map

{'Bruce': ObjectId('665d8ff66ce65f51c7e17e2a'),
 'Clint': ObjectId('665d8ff66ce65f51c7e17e2b'),
 'Natasha': ObjectId('665d8ff66ce65f51c7e17e2c'),
 'Steve': ObjectId('665d8ff66ce65f51c7e17e2d'),
 'Tony': ObjectId('665d8ff66ce65f51c7e17e2e')}

# Semester Data

In [7]:
df_evaluation = pd.read_excel("Student-Data.xlsx", "Semester_Data").convert_dtypes().replace({pd.NA:None})
df_evaluation.head()

,firstname,semester_name,niveau,last_progress_talk,last_report
0,Bruce,FS24,A,FS24,HS23
1,Clint,FS24,B,FS24,HS23
2,Natasha,FS24,B,FS24,HS23
3,Steve,FS24,A,FS24,HS23
4,Tony,FS24,A,HS23,HS23


In [8]:
# Create dictionary with all the important information, including ObjectID-reference to the student
evaluation_data = {
    (row.firstname, row.semester_name): dict(
        student_id=student_map[row.firstname],
        **{
            k:v for k,v in
            row.to_dict().items()
            if k != "firstname"
        },
        final_assessment=None, # will be needed later
        text=None, # will be needed later
    )
    for _, row in df_evaluation.iterrows()
}
list(evaluation_data.values())[0]

{'student_id': ObjectId('665d8ff66ce65f51c7e17e2a'),
 'semester_name': 'FS24',
 'niveau': 'A',
 'last_progress_talk': 'FS24',
 'last_report': 'HS23',
 'final_assessment': None,
 'text': None}

In [9]:
# Load Entries to MongoDB
collection = db["Semester_Data"]
result = collection.insert_many(evaluation_data.values())

# Mapping: (student_id, semester) to ObjectIDs of Evaluations
evaluation_map = {
    k: r
    for k,r in zip(evaluation_data.keys(), result.inserted_ids)
}

In [10]:
evaluation_map

{('Bruce', 'FS24'): ObjectId('665d8ff86ce65f51c7e17e2f'),
 ('Clint', 'FS24'): ObjectId('665d8ff86ce65f51c7e17e30'),
 ('Natasha', 'FS24'): ObjectId('665d8ff86ce65f51c7e17e31'),
 ('Steve', 'FS24'): ObjectId('665d8ff86ce65f51c7e17e32'),
 ('Tony', 'FS24'): ObjectId('665d8ff86ce65f51c7e17e33'),
 ('Bruce', 'HS23'): ObjectId('665d8ff86ce65f51c7e17e34'),
 ('Clint', 'HS23'): ObjectId('665d8ff86ce65f51c7e17e35'),
 ('Natasha', 'HS23'): ObjectId('665d8ff86ce65f51c7e17e36'),
 ('Steve', 'HS23'): ObjectId('665d8ff86ce65f51c7e17e37'),
 ('Tony', 'HS23'): ObjectId('665d8ff86ce65f51c7e17e38'),
 ('Bruce', 'FS23'): ObjectId('665d8ff86ce65f51c7e17e39'),
 ('Clint', 'FS23'): ObjectId('665d8ff86ce65f51c7e17e3a'),
 ('Natasha', 'FS23'): ObjectId('665d8ff86ce65f51c7e17e3b'),
 ('Steve', 'FS23'): ObjectId('665d8ff86ce65f51c7e17e3c'),
 ('Tony', 'FS23'): ObjectId('665d8ff86ce65f51c7e17e3d')}

# Assessments

In [11]:
df_assessment = pd.read_excel(
    "Student-Data.xlsx",
    "Assessments_Data",
    converters=dict(date=pd.to_datetime),
).convert_dtypes().replace({pd.NA:None})
df_assessment.head()

,firstname,semester_name,author,AktivTeilnehmen,AktivTeilnehmenNotizen,LeistungZeigen,LeistungZeigenNotizen,AufmerksamSein,AufmerksamSeinNotizen,SchulinhalteMerken,SchulinhalteMerkenNotizen,SchulinhalteAbrufen,SchulinhalteAbrufenNotizen,date
0,Bruce,HS23,Daniela,gut,"insbesondere bei Fächer, die ihn interessieren",gut,teilweise könnte er noch etwas mehr Leistung z...,sehr gut,None,gut,None,teilweise,"ab und zu fällt es Bruce schwer, bei Prüfungen...",2023-06-01 09:42:14.695000+00:00
1,Bruce,HS23,Yves,sehr gut,Bruce hat viele schulische Interessen. Insbeso...,gut,None,gut,None,noch nicht,None,teilweise,"manchmal hat er Schwierigkeiten, bei Prüfungen...",2023-06-01 09:42:13.695000+00:00
2,Bruce,FS24,Daniela,sehr gut,None,gut,Bruce hat sich seit dem letzten Semester sehr ...,gut,None,gut,None,gut,None,2024-06-01 09:42:13.695000+00:00
3,Bruce,FS24,Yves,sehr gut,None,gut,None,sehr gut,None,teilweise,None,gut,None,2024-06-01 09:42:14.695000+00:00


In [12]:
# Create dictionary with all the important information, including ObjectID - reference to the student
assessment_data = []
assessment_critera = {
    "AktivTeilnehmen",
    "LeistungZeigen",
    "AufmerksamSein",
    "SchulinhalteMerken",
    "SchulinhalteAbrufen",
}
for _, row in df_assessment.iterrows():
    rowdict = row.to_dict()
    assessment = {
        criterium: dict(
            assessment=rowdict.pop(criterium),
            notes=rowdict.pop(criterium+"Notizen"),
        ) for criterium in assessment_critera
    }
    # Rule: All Assessment-Criteria are in UpperCase and we only want to handle these
    assert not any(k[0].isupper() for k in rowdict)

    rowdict.pop("firstname")
    rowdict.pop("semester_name")
    
    
    assessment_data.append(dict(
        student_id=student_map[row.firstname],
        semester_id=evaluation_map[row.firstname, row.semester_name],
        allgemeines_lernen=assessment,
        **rowdict,
    ))

assessment_data[0]

{'student_id': ObjectId('665d8ff66ce65f51c7e17e2a'),
 'semester_id': ObjectId('665d8ff86ce65f51c7e17e34'),
 'allgemeines_lernen': {'SchulinhalteAbrufen': {'assessment': 'teilweise',
   'notes': 'ab und zu fällt es Bruce schwer, bei Prüfungen zu zeigen, was er kann.'},
  'LeistungZeigen': {'assessment': 'gut',
   'notes': 'teilweise könnte er noch etwas mehr Leistung zeigen, wenn er rechtzeit lernen würde'},
  'SchulinhalteMerken': {'assessment': 'gut', 'notes': None},
  'AktivTeilnehmen': {'assessment': 'gut',
   'notes': 'insbesondere bei Fächer, die ihn interessieren'},
  'AufmerksamSein': {'assessment': 'sehr gut', 'notes': None}},
 'author': 'Daniela',
 'date': Timestamp('2023-06-01 09:42:14.695000+0000', tz='UTC')}

In [13]:
collection = db["Assessment_Data"]
result = collection.insert_many(assessment_data)